"""
MIT License

Copyright (c) 2021 porteratzo

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.
"""

Introduction

This document serves as a tutorial for using the treetool tool, a software for detecting trees in a point cloud and measuring their diameter at breast height (1.3 m). This document seeks to demonstrate the operation of treetool, whether used as a stand-alone application or integrated as a package with other applications.

Usage guide

Below we describe our demo notebook contained in the QuickDemo.ipynb file. This notebook illustrates the operation and use of our software, from loading a point cloud, viewing it, processing it with our algorithm and saving the results.


Load the libraries that we will use and had previously installed

In [1]:
import pclpy
import numpy as np
import treetool.seg_tree as seg_tree
import treetool.utils as utils
import treetool.tree_tool as tree_tool
import pandas as pd
from scipy.optimize import linear_sum_assignment
import matplotlib.pyplot as plt

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


Load the point cloud from a .pcd using pclpy, we use our function seg_tree which contains many helper functions such as voxelize to down sample our point cloud and visualize using our Visualization function built on open3d. 


In [2]:
import laspy
file_directory = r'data/plot_100_3_MLS.las'
#file_directory = r'data/plot_10.las'
with laspy.open(file_directory, 'r') as f:
    data = f.read()

In [3]:
data.vlrs

[]

In [4]:
points = utils.scaled_dimensions(data)

In [5]:
PointCloud = pclpy.pcl.PointCloud.PointXYZ()
PointCloudV = seg_tree.voxelize(points,0.04)
utils.open3dpaint(PointCloudV, reduce_for_vis = True  , voxel_size = 0.04)

Tree tool is our main class that contains the routines for tree detection and DBH extraction

In [6]:
My_treetool = tree_tool.treetool(PointCloudV)

Our tree top object has a series of functions that are performed to obtain DBH and tree detection.

In [7]:
My_treetool.step_1_remove_floor()

#Obtained attributes:
#non_ground_cloud: All points in the point cloud that don't belong to the ground
#ground_cloud: All points in the point cloud that belong to the ground
utils.open3dpaint([My_treetool.non_ground_cloud,My_treetool.ground_cloud],reduce_for_vis = True  , voxel_size = 0.1)

Set Algorithm Parameters

Run main process

In [8]:
#Get point normals for filtering

#Obtained attributes:
#non_filtered_points: Same as non_ground_cloud
#non_filtered_normals: Normals of points in non_filtered_points
#filtered_points: Points that pass the normal filter
#filtered_normals: Normals of points that pass the normal filter
My_treetool.step_2_normal_filtering(verticality_threshold=0.04, curvature_threshold=0.06, search_radius=0.2)
#utils.open3dpaint([My_treetool.non_ground_cloud.xyz, My_treetool.non_filtered_points.xyz + My_treetool.non_filtered_normals * 0.1, My_treetool.non_filtered_points.xyz + My_treetool.non_filtered_normals * 0.2], reduce_for_vis = True , voxel_size = 0.1)

#utils.open3dpaint([My_treetool.filtered_points.xyz, My_treetool.filtered_points.xyz + My_treetool.filtered_normals * 0.05, My_treetool.filtered_points.xyz + My_treetool.filtered_normals * 0.1], reduce_for_vis = True , voxel_size = 0.1)
utils.open3dpaint([My_treetool.non_filtered_points.xyz + 20, My_treetool.filtered_points.xyz, ], pointsize=2)

In [9]:
My_treetool.step_3_euclidean_clustering(tolerance=0.4, min_cluster_size=40, max_cluster_size=6000000)

#Obtained attributes:
#cluster_list: List of all clusters obtained with Euclidean Clustering

utils.open3dpaint(My_treetool.cluster_list,reduce_for_vis = True  , voxel_size = 0.1)

In [10]:
#Group stem segments
My_treetool.step_4_group_stems(max_distance=0.4)

#Obtained attributes:
#complete_Stems: List of all complete stems obtained by joining clusters belonging to the same tree
            
utils.open3dpaint(My_treetool.complete_Stems,reduce_for_vis = True  , voxel_size = 0.1)

In [11]:
My_treetool.step_5_get_ground_level_trees(lowstems_height=5, cutstems_height=5)

#Obtained attributes:
#low_stems: List of all stems truncated to the specified height

utils.open3dpaint(My_treetool.low_stems,reduce_for_vis = True  , voxel_size = 0.1)

In [12]:
My_treetool.step_6_get_cylinder_tree_models(search_radius=0.1)

#Obtained attributes:
#finalstems: List of Dictionaries with two keys 'tree' which contains the points used to fit the cylinder model and 'model' which contains the cylinder model parameters
#visualization_cylinders: List of the pointclouds that represent the tree modeled with a cylinder

utils.open3dpaint([i['tree'] for i in My_treetool.finalstems] + My_treetool.visualization_cylinders,reduce_for_vis = True  , voxel_size = 0.1)
     

In [13]:
My_treetool.step_7_ellipse_fit()

#Obtained attributes:
#Three new keys in our finalstems dictionaries:
#final_diameter: Final DBH of every tree
#cylinder_diameter: DBH obtained with cylinder fitting
#ellipse_diameter;DBH obtained with Ellipse fitting

In [14]:
cloud_match = [i['tree'] for i in My_treetool.finalstems]+[i for i in My_treetool.visualization_cylinders]
utils.open3dpaint(cloud_match+[PointCloudV], voxel_size = 0.1, pointsize=2)

In [15]:
def getdata(treetool_obj):
        """
        Save a csv with XYZ and DBH of each detected tree

        Args:
            savelocation : str
                path to save file

        Returns:
            None
        """
        tree_model_info = [i['model'] for i in treetool_obj.finalstems]
        tree_diameter_info = [i['final_diameter'] for i in treetool_obj.finalstems]

        data = {'X': [], 'Y': [], 'Z': [], 'DBH': []}
        for i, j in zip(tree_model_info, tree_diameter_info):
            data['X'].append(i[0])
            data['Y'].append(i[1])
            data['Z'].append(i[2])
            data['DBH'].append(j)
        return data

In [16]:
tree_data = getdata(My_treetool)
scales_tree_data = utils.Iscaled_dimensions(data, tree_data)
data_to_save = tree_data.copy()
data_to_save['X'] = scales_tree_data[:,0]
data_to_save['Y'] = scales_tree_data[:,1]
data_to_save['Z'] = scales_tree_data[:,2]

In [128]:
import os
save_location = 'results/myresults.csv'
os.makedirs(os.path.dirname(save_location), exist_ok=True)

pd.DataFrame.from_dict(data_to_save).to_csv(save_location)